In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json
/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json


In [2]:
import json
import pandas as pd 


Create DataFrame from the dataset

The JSON is not formated Properly

Creating a Function to Format JSON

The Data Format which will be used is

In [3]:
def format_json(file):
    with open(file,"r") as f:
        text = f.read()
    data = text.split("\n")
    
    #print(data[-50:])
    data = [json.loads(e) for e in data if e != ""]
    res = {
        "data": data
    }
    return res
    
### Get the data    
data_one = format_json("../input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json")
data_two = format_json("../input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json")


In [4]:
###  Data Transformed to Data Frame 
def trainsform_data(data):
    data = data['data']
    keys = data[0].keys()
    
    try:
        res = {k: [d[k] for d in data] for k in keys}
        res = pd.DataFrame(res)
    except:
        print("Error the Dict Keys are inconsistent")
        print("Every dict element in the list does not have same number of keys")
        res = None
    return res

##########################################################################
df_one = trainsform_data(data_one)
df_two = trainsform_data(data_two)


print(df_one.shape,df_two.shape)

(26709, 3) (28619, 3)


In [5]:
df_one.describe()
df_one.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


**Preprocessing the headlines**

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenize = Tokenizer(oov_token="<OOV>")

In [7]:
tokenize.fit_on_texts(df_one['headline'])

In [8]:
word_index = tokenize.word_index
print(f"Toltal number of words in the dataset = {len(word_index)}")

# print(df_one['headline'])

sentences = []
for i in range(0,len(df_one)):
    sentences.append(df_one['headline'][i])
# print(sentences)

sequence = tokenize.texts_to_sequences(list(df_one['headline']))
# print(sequence)
padding = pad_sequences(sequence, padding='pre')

index=4
print(df_one['headline'][index])
print(padding[index])
print(padding.shape)

Toltal number of words in the dataset = 29657
j.k. rowling wishes snape happy birthday in the most magical way
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0   767   719  4720   908 10737   623   594     5
     4    95  1309    92]
(26709, 40)
